## Rhetorical relations classification used in tree building: Step 3. BiMPM

Prepare data and model-related scripts.

Evaluate models.

Make and evaluate an ansemble for BiMPM and feature rich model.

Output:
 - ``models/relation_predictor_bimpm/*``

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import pickle
from utils.file_reading import read_edus, read_gold, read_negative, read_annotation

### Make a directory

In [ ]:
MODEL_PATH = 'models/label_predictor_bimpm'
! mkdir $MODEL_PATH

TRAIN_FILE_PATH = os.path.join(MODEL_PATH, 'nlabel_cf_train.tsv')
DEV_FILE_PATH = os.path.join(MODEL_PATH, 'nlabel_cf_dev.tsv')
TEST_FILE_PATH = os.path.join(MODEL_PATH, 'nlabel_cf_test.tsv')

### Prepare train/test sets 

In [ ]:
IN_PATH = 'data_labeling'

train_samples = pd.read_pickle(os.path.join(IN_PATH, 'train_samples.pkl'))
dev_samples = pd.read_pickle(os.path.join(IN_PATH, 'dev_samples.pkl'))
test_samples = pd.read_pickle(os.path.join(IN_PATH, 'test_samples.pkl'))

In [ ]:
counts = train_samples['relation'].value_counts(normalize=False).values
NUMBER_CLASSES = len(counts)
print("number of classes:", NUMBER_CLASSES)
print("class weights:")
np.round(counts.min() / counts, decimals=6)

In [ ]:
train_samples = train_samples.reset_index()
train_samples[['relation', 'snippet_x', 'snippet_y', 'index']].to_csv(TRAIN_FILE_PATH, sep='\t', header=False, index=False)

dev_samples = dev_samples.reset_index()
dev_samples[['relation', 'snippet_x', 'snippet_y', 'index']].to_csv(DEV_FILE_PATH, sep='\t', header=False, index=False)

test_samples = test_samples.reset_index()
test_samples[['relation', 'snippet_x', 'snippet_y', 'index']].to_csv(TEST_FILE_PATH, sep='\t', header=False, index=False)

### Modify model

Note: Core code for ``bimpm_custom_package`` is in ``2.3_structure_classification_custom_bimpm.ipynb``

In [ ]:
%%writefile models/bimpm_custom_package/model/multiclass_bimpm.py

"""
BiMPM (Bilateral Multi-Perspective Matching) model implementation.
"""

from typing import Dict, Optional, List, Any

from overrides import overrides
import torch
import numpy
from allennlp.common.checks import check_dimensions_match
from allennlp.data import Vocabulary
from allennlp.modules import FeedForward, Seq2SeqEncoder, Seq2VecEncoder, TextFieldEmbedder
from allennlp.models.model import Model
from allennlp.nn import InitializerApplicator, RegularizerApplicator
from allennlp.nn import util
from allennlp.training.metrics import CategoricalAccuracy, F1Measure

from allennlp.modules.bimpm_matching import BiMpmMatching

from allennlp.nn.util import get_text_field_mask
import torch.nn.functional as F


@Model.register("multiclass_bimpm")
class BiMpm(Model):
    """
    This ``Model`` augments with additional features the BiMPM model described in `Bilateral Multi-Perspective 
    Matching for Natural Language Sentences <https://arxiv.org/abs/1702.03814>`_ by Zhiguo Wang et al., 2017.
    implemented in https://github.com/galsang/BIMPM-pytorch>`_.
    Additional features are added before the feedforward classifier.
    """
    def __init__(self, vocab: Vocabulary,
                 text_field_embedder: TextFieldEmbedder,
                 matcher_word: BiMpmMatching,
                 encoder1: Seq2SeqEncoder,
                 matcher_forward1: BiMpmMatching,
                 matcher_backward1: BiMpmMatching,
                 encoder2: Seq2SeqEncoder,
                 matcher_forward2: BiMpmMatching,
                 matcher_backward2: BiMpmMatching,
                 aggregator: Seq2VecEncoder,
                 classifier_feedforward: FeedForward,
                 dropout: float = 0.1,
                 class_weights: list = [],
                 encode_together: bool = False,
                 encode_lstm: bool = True,
                 initializer: InitializerApplicator = InitializerApplicator(),
                 regularizer: Optional[RegularizerApplicator] = None) -> None:
        super(BiMpm, self).__init__(vocab, regularizer)

        self.text_field_embedder = text_field_embedder

        self.matcher_word = matcher_word

        self.encoder1 = encoder1
        self.matcher_forward1 = matcher_forward1
        self.matcher_backward1 = matcher_backward1

        self.encoder2 = encoder2
        self.matcher_forward2 = matcher_forward2
        self.matcher_backward2 = matcher_backward2

        self.aggregator = aggregator
        
        self.encode_together = encode_together
        self.encode_lstm = encode_lstm
        
        matching_dim = self.matcher_word.get_output_dim()
        
        if self.encode_lstm:
            matching_dim += self.matcher_forward1.get_output_dim(
            ) + self.matcher_backward1.get_output_dim(
            ) + self.matcher_forward2.get_output_dim(
            ) + self.matcher_backward2.get_output_dim(
            )
            
        check_dimensions_match(matching_dim, self.aggregator.get_input_dim(),
                       "sum of dim of all matching layers", "aggregator input dim")

        self.classifier_feedforward = classifier_feedforward

        self.dropout = torch.nn.Dropout(dropout)
        
        if class_weights:
            self.class_weights = class_weights
        else:
            self.class_weights = [1.] * self.classifier_feedforward.get_output_dim()
            
        self.metrics = {"accuracy": CategoricalAccuracy()}
        
        for _class in range(len(self.class_weights)):
            self.metrics.update({
                f"f1_rel{_class}": F1Measure(_class),
            })

        self.loss = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(self.class_weights))

        initializer(self)

    @overrides
    def forward(self,  # type: ignore
                premise: Dict[str, torch.LongTensor],
                hypothesis: Dict[str, torch.LongTensor],
                label: torch.LongTensor = None,
                metadata: List[Dict[str, Any]] = None  # pylint:disable=unused-argument
               ) -> Dict[str, torch.Tensor]:

        """
        Parameters
        ----------
        premise : Dict[str, torch.LongTensor]
            The premise from a ``TextField``
        hypothesis : Dict[str, torch.LongTensor]
            The hypothesis from a ``TextField``
        label : torch.LongTensor, optional (default = None)
            The label for the pair of the premise and the hypothesis
        metadata : ``List[Dict[str, Any]]``, optional, (default = None)
            Additional information about the pair
        Returns
        -------
        An output dictionary consisting of:
        logits : torch.FloatTensor
            A tensor of shape ``(batch_size, num_labels)`` representing unnormalised log
            probabilities of the entailment label.
        loss : torch.FloatTensor, optional
            A scalar loss to be optimised.
        """
        
        def encode_pair(x1, x2, mask1=None, mask2=None):
            _joined_pair: Dict[str, torch.LongTensor] = {}
            
            for key in premise.keys():
                bsz = premise[key].size(0)
                x1_len, x2_len = premise[key].size(1), hypothesis[key].size(1)
                sep = torch.empty([bsz, 1], dtype=torch.long, device=premise[key].device)
                sep.data.fill_(0) # 2 is the id for </s>
                
                x = torch.cat([premise[key], hypothesis[key]], dim=1)
                _joined_pair[key] = x
                
            x_output = self.dropout(self.text_field_embedder(_joined_pair))
            return x_output[:, :x1_len], x_output[:, -x2_len:], mask1, mask2

        mask_premise = util.get_text_field_mask(premise)
        mask_hypothesis = util.get_text_field_mask(hypothesis)
        
        if self.encode_together:
            embedded_premise, embedded_hypothesis, _, _ = encode_pair(premise, hypothesis)
        else:
            embedded_premise = self.dropout(self.text_field_embedder(premise))
            embedded_hypothesis = self.dropout(self.text_field_embedder(hypothesis))

        # embedding and encoding of the premise
        encoded_premise1 = self.dropout(self.encoder1(embedded_premise, mask_premise))
        encoded_premise2 = self.dropout(self.encoder2(encoded_premise1, mask_premise))

        # embedding and encoding of the hypothesis
        encoded_hypothesis1 = self.dropout(self.encoder1(embedded_hypothesis, mask_hypothesis))
        encoded_hypothesis2 = self.dropout(self.encoder2(encoded_hypothesis1, mask_hypothesis))
        
        matching_vector_premise: List[torch.Tensor] = []
        matching_vector_hypothesis: List[torch.Tensor] = []

        def add_matching_result(matcher, encoded_premise, encoded_hypothesis):
            # utility function to get matching result and add to the result list
            matching_result = matcher(encoded_premise, mask_premise, encoded_hypothesis, mask_hypothesis)
            matching_vector_premise.extend(matching_result[0])
            matching_vector_hypothesis.extend(matching_result[1])

        # calculate matching vectors from word embedding, first layer encoding, and second layer encoding
        add_matching_result(self.matcher_word, embedded_premise, embedded_hypothesis)
        half_hidden_size_1 = self.encoder1.get_output_dim() // 2
        add_matching_result(self.matcher_forward1,
                            encoded_premise1[:, :, :half_hidden_size_1],
                            encoded_hypothesis1[:, :, :half_hidden_size_1])
        add_matching_result(self.matcher_backward1,
                            encoded_premise1[:, :, half_hidden_size_1:],
                            encoded_hypothesis1[:, :, half_hidden_size_1:])

        half_hidden_size_2 = self.encoder2.get_output_dim() // 2
        add_matching_result(self.matcher_forward2,
                            encoded_premise2[:, :, :half_hidden_size_2],
                            encoded_hypothesis2[:, :, :half_hidden_size_2])
        add_matching_result(self.matcher_backward2,
                            encoded_premise2[:, :, half_hidden_size_2:],
                            encoded_hypothesis2[:, :, half_hidden_size_2:])

        # concat the matching vectors
        matching_vector_cat_premise = self.dropout(torch.cat(matching_vector_premise, dim=2))
        matching_vector_cat_hypothesis = self.dropout(torch.cat(matching_vector_hypothesis, dim=2))

        # aggregate the matching vectors
        aggregated_premise = self.dropout(self.aggregator(matching_vector_cat_premise, mask_premise))
        aggregated_hypothesis = self.dropout(self.aggregator(matching_vector_cat_hypothesis, mask_hypothesis))

        # encode additional information
        #batch_size, _ = aggregated_premise.size()
        #encoded_meta = metadata.float().view(batch_size, -1)
        
        # the final forward layer
        logits = self.classifier_feedforward(torch.cat([aggregated_premise, aggregated_hypothesis], dim=-1))
        probs = torch.nn.functional.softmax(logits, dim=-1)

        output_dict = {'logits': logits, "probs": probs}
        
        if label is not None:
            loss = self.loss(logits, label)
            for metric in self.metrics.values():
                metric(logits, label)
            output_dict["loss"] = loss

        return output_dict

    @overrides
    def decode(self, output_dict: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        """
        Converts indices to string labels, and adds a ``"label"`` key to the result.
        """
        predictions = output_dict["probs"].cpu().data.numpy()
        argmax_indices = numpy.argmax(predictions, axis=-1)
        labels = [self.vocab.get_token_from_index(x, namespace="labels")
                  for x in argmax_indices]
        output_dict['label'] = labels
        return output_dict

    @overrides
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        metrics = {"accuracy": self.metrics["accuracy"].get_metric(reset=reset)}
        
        for _class in range(len(self.class_weights)):
            metrics.update({
                f"f1_rel{_class}": self.metrics[f"f1_rel{_class}"].get_metric(reset=reset)[2],
            })
        
        metrics["f1_macro"] = numpy.mean([metrics[f"f1_rel{_class}"] for _class in range(len(self.class_weights))])
        return metrics


### 2. Generate config files

#### ELMo 

In [ ]:
%%writefile $MODEL_PATH/config_elmo.json

// Configuration for a sentence matching model based on:
//   Wang, Zhiguo, Wael Hamza, and Radu Florian. "Bilateral multi-perspective matching for natural language sentences."
//   Proceedings of the 26th International Joint Conference on Artificial Intelligence. 2017.

{
  "dataset_reader": {
    "type": "quora_paraphrase",
    "lazy": false,
    "tokenizer": {
      "type": "word",
      "word_splitter": {
        "type": "just_spaces"
      }
    },
    "token_indexers": {
      "token_characters": {
        "type": "characters",
        "min_padding_length": 30,
      },
      "elmo": {
        "type": "elmo_characters"
     }
    }
  },
  "train_data_path": "label_predictor_lstm/nlabel_cf_train.tsv",
  "validation_data_path": "label_predictor_lstm/nlabel_cf_dev.tsv",
  "test_data_path": "label_predictor_lstm/nlabel_cf_test.tsv",
  "model": {
    "type": "multiclass_bimpm",
    "dropout": 0.5,
    "class_weights": [
        0.025041, 0.028714, 0.074134, 0.111786, 0.123656, 0.126722,
        0.133333, 0.154104, 0.168498, 0.169118, 0.180747, 0.187373,
        0.234097, 0.235897, 0.328571, 0.339483, 0.355212, 0.516854,
        0.578616, 0.707692, 0.859813, 0.87619 , 1.0      ],
    "encode_together": false,
    "text_field_embedder": {
        "token_embedders": {
            "elmo": {
                    "type": "elmo_token_embedder",
                    "options_file": "rsv_elmo/options.json",
                    "weight_file": "rsv_elmo/model.hdf5",
                    "do_layer_norm": false,
                    "dropout": 0.1
            },
            "token_characters": {
                "type": "character_encoding",
                "dropout": 0.1,
                "embedding": {
                    "embedding_dim": 20,
                    "padding_index": 0,
                },
                "encoder": {
                    "type": "gru",
                    "input_size": 20,
                    "hidden_size": 50,
                    "num_layers": 1,
                    "bidirectional": true,
                },
            },
      }
    },
    "matcher_word": {
      "is_forward": true,
      "hidden_dim": 1024+100,
      "num_perspectives": 10,
      "with_full_match": false
    },
    "encoder1": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 1024+100,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward1": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward1": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "encoder2": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 400,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward2": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward2": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "aggregator":{
      "type": "lstm",
      "bidirectional": true,
      "input_size": 264,
      "hidden_size": 100,
      "num_layers": 1,
    },
    "classifier_feedforward": {
      "input_dim": 400,
      "num_layers": 1,
      "hidden_dims": [23],
      "activations": ["linear"],
      "dropout": [0.0]
    },
    "initializer": [
      [".*linear_layers.*weight", {"type": "xavier_normal"}],
      [".*linear_layers.*bias", {"type": "constant", "val": 0}],
      [".*weight_ih.*", {"type": "xavier_normal"}],
      [".*weight_hh.*", {"type": "orthogonal"}],
      [".*bias.*", {"type": "constant", "val": 0}],
      [".*matcher.*match_weights.*", {"type": "kaiming_normal"}]
    ],
  },
  "iterator": {
    "type": "bucket",
    "padding_noise": 0,
    "sorting_keys": [["premise", "num_tokens"], ["hypothesis", "num_tokens"]],
    "batch_size": 4
  },
  "trainer": {
    "num_epochs": 200,
    "cuda_device": 1,
    "shuffle": true,
    "optimizer": {
      "type": "adam",
      "lr": 0.001
    },
    "type":"callback",
    "callbacks":[
        {
            "type": "validate"
        },
        {
            "type": "checkpoint",
            "checkpointer":{
                "num_serialized_models_to_keep":1
            }
        },
        {
            "type": "gradient_norm_and_clip", 
            "grad_norm": 5.0
        },
        {
            "type": "track_metrics",
            "patience": 20,
            "validation_metric": "+f1_macro"
        },
        {
            "type": "log_metrics_to_wandb"
        }
    ],
  }
}

#### ELMo + fasttext 

In [ ]:
%%writefile $MODEL_PATH/config_elmo_fasttext.json

// Configuration for a sentence matching model based on:
//   Wang, Zhiguo, Wael Hamza, and Radu Florian. "Bilateral multi-perspective matching for natural language sentences."
//   Proceedings of the 26th International Joint Conference on Artificial Intelligence. 2017.

{
  "dataset_reader": {
    "type": "quora_paraphrase",
    "lazy": false,
    "tokenizer": {
      "type": "word",
      "word_splitter": {
        "type": "just_spaces"
      }
    },
    "token_indexers": {
      "token_characters": {
        "type": "characters",
        "min_padding_length": 30,
      },
      "elmo": {
        "type": "elmo_characters"
      },
      "tokens": {
        "type": "single_id",
        "lowercase_tokens": true
      },
    }
  },
  "train_data_path": "label_predictor_bimpm/nlabel_cf_train.tsv",
  "validation_data_path": "label_predictor_bimpm/nlabel_cf_dev.tsv",
  "test_data_path": "label_predictor_bimpm/nlabel_cf_test.tsv",
  "model": {
    "type": "multiclass_bimpm",
    "dropout": 0.5,
    "class_weights": [
        0.025041, 0.028714, 0.074134, 0.111786, 0.123656, 0.126722,
        0.133333, 0.154104, 0.168498, 0.169118, 0.180747, 0.187373,
        0.234097, 0.235897, 0.328571, 0.339483, 0.355212, 0.516854,
        0.578616, 0.707692, 0.859813, 0.87619 , 1.0      ],
    "encode_together": false,
    "text_field_embedder": {
        "token_embedders": {
            "tokens": {
                "type": "embedding",
                "embedding_dim": 300,
                "pretrained_file": "ft_native_300_ru_wiki_lenta_nltk_wordpunct_tokenize.vec",
                "trainable": false
            },
            "elmo": {
                    "type": "elmo_token_embedder",
                    "options_file": "rsv_elmo/options.json",
                    "weight_file": "rsv_elmo/model.hdf5",
                    "do_layer_norm": false,
                    "dropout": 0.1
            },
            "token_characters": {
                "type": "character_encoding",
                "dropout": 0.1,
                "embedding": {
                    "embedding_dim": 20,
                    "padding_index": 0,
                },
                "encoder": {
                    "type": "gru",
                    "input_size": 20,
                    "hidden_size": 50,
                    "num_layers": 1,
                    "bidirectional": true,
                },
            },
      }
    },
    "matcher_word": {
      "is_forward": true,
      "hidden_dim": 1024+100+300,
      "num_perspectives": 10,
      "with_full_match": false
    },
    "encoder1": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 1024+100+300,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward1": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward1": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "encoder2": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 400,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward2": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward2": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "aggregator":{
      "type": "lstm",
      "bidirectional": true,
      "input_size": 264,
      "hidden_size": 100,
      "num_layers": 1,
    },
    "classifier_feedforward": {
      "input_dim": 400,
      "num_layers": 1,
      "hidden_dims": [23],
      "activations": ["linear"],
      "dropout": [0.0]
    },
    "initializer": [
      [".*linear_layers.*weight", {"type": "xavier_normal"}],
      [".*linear_layers.*bias", {"type": "constant", "val": 0}],
      [".*weight_ih.*", {"type": "xavier_normal"}],
      [".*weight_hh.*", {"type": "orthogonal"}],
      [".*bias.*", {"type": "constant", "val": 0}],
      [".*matcher.*match_weights.*", {"type": "kaiming_normal"}]
    ],
  },
  "iterator": {
    "type": "bucket",
    "padding_noise": 0,
    "sorting_keys": [["premise", "num_tokens"], ["hypothesis", "num_tokens"]],
    "batch_size": 4
  },
  "trainer": {
    "num_epochs": 200,
    "cuda_device": 1,
    "shuffle": true,
    "optimizer": {
      "type": "adam",
      "lr": 0.001
    },
    "type":"callback",
    "callbacks":[
        {
            "type": "validate"
        },
        {
            "type": "checkpoint",
            "checkpointer":{
                "num_serialized_models_to_keep":1
            }
        },
        {
            "type": "gradient_norm_and_clip", 
            "grad_norm": 5.0
        },
        {
            "type": "track_metrics",
            "patience": 20,
            "validation_metric": "+f1_macro"
        },
        {
            "type": "log_metrics_to_wandb"
        }
    ],
  }
}

### 3. Scripts for training/prediction 

#### Option 1. Directly from the config

Train a model

In [ ]:
%%writefile models/train_label_predictor.sh
# usage:
# $ cd models 
# $ sh train_label_predictor.sh {bert|elmo} result_30

export METHOD=${1}
export RESULT_DIR=${2}
export DEV_FILE_PATH="nlabel_cf_dev.tsv"
export TEST_FILE_PATH="nlabel_cf_test.tsv"

rm -r label_predictor_bimpm/${RESULT_DIR}/
allennlp train -s label_predictor_bimpm/${RESULT_DIR}/ label_predictor_bimpm/config_${METHOD}.json \
    --include-package customization_package
allennlp predict --use-dataset-reader --silent \
    --output-file label_predictor_bimpm/${RESULT_DIR}/predictions_dev.json label_predictor_bimpm/${RESULT_DIR}/model.tar.gz label_predictor_lstm/${DEV_FILE_PATH} \
    --include-package customization_package \
    --predictor textual-entailment
allennlp predict --use-dataset-reader --silent \
    --output-file label_predictor_bimpm/${RESULT_DIR}/predictions_test.json label_predictor_bimpm/${RESULT_DIR}/model.tar.gz label_predictor_lstm/${TEST_FILE_PATH} \
    --include-package customization_package \
    --predictor textual-entailment

Predict on dev&test

In [ ]:
%%writefile models/eval_label_predictor.sh
# usage:
# $ cd models 
# $ sh train_label_predictor.sh {bert|elmo} result_30

export METHOD=${1}
export RESULT_DIR=${2}
export DEV_FILE_PATH="nlabel_cf_dev.tsv"
export TEST_FILE_PATH="nlabel_cf_test.tsv"

allennlp predict --use-dataset-reader --silent \
    --output-file label_predictor_bimpm/${RESULT_DIR}/predictions_dev.json label_predictor_bimpm/${RESULT_DIR}/model.tar.gz label_predictor_bimpm/${DEV_FILE_PATH} \
    --include-package customization_package \
    --predictor textual-entailment
allennlp predict --use-dataset-reader --silent \
    --output-file label_predictor_bimpm/${RESULT_DIR}/predictions_test.json label_predictor_bimpm/${RESULT_DIR}/model.tar.gz label_predictor_bimpm/${TEST_FILE_PATH} \
    --include-package customization_package \
    --predictor textual-entailment

(optional) predict on train

In [ ]:
%%writefile models/eval_label_predictor_train.sh
# usage:
# $ cd models 
# $ sh eval_label_predictor_train.sh {bert|elmo} result_30

export METHOD=${1}
export RESULT_DIR=${2}
export TEST_FILE_PATH="nlabel_cf_train.tsv"

allennlp predict --use-dataset-reader --silent \
    --output-file label_predictor_bimpm/${RESULT_DIR}/predictions_train.json label_predictor_bimpm/${RESULT_DIR}/model.tar.gz label_predictor_bimpm/${TEST_FILE_PATH} \
    --include-package customization_package \
    --predictor textual-entailment

#### Option 2. Using wandb for parameters adjustment

In [ ]:
%%writefile models/wandb_label_predictor1.yaml

name: label_predictor_stacked
program: wandb_allennlp # this is a wrapper console script around allennlp commands. It is part of wandb-allennlp
method: bayes
## Do not for get to use the command keyword to specify the following command structure
command:
  - ${program} #omit the interpreter as we use allennlp train command directly
  - "--subcommand=train"
  - "--include-package=bimpm_custom_package" # add all packages containing your registered classes here
  - "--config_file=label_predictor_bimpm/config_elmo.json"
  - ${args}
metric:
    name: best_f1_macro
    goal: maximize
parameters:
    iterator.batch_size:
        values: [4,]
    model.encode_together:
        values: ["true", ]
    trainer.optimizer.lr:
        values: [0.001,]
    model.dropout:
        values: [0.5]


In [ ]:
%%writefile models/wandb_label_predictor2.yaml

name: label_predictor_stacked
program: wandb_allennlp # this is a wrapper console script around allennlp commands. It is part of wandb-allennlp
method: bayes
## Do not for get to use the command keyword to specify the following command structure
command:
  - ${program} #omit the interpreter as we use allennlp train command directly
  - "--subcommand=train"
  - "--include-package=bimpm_custom_package" # add all packages containing your registered classes here
  - "--config_file=label_predictor_bimpm/config_elmo_fasttext.json"
  - ${args}
metric:
    name: best_f1_macro
    goal: maximize
parameters:
    iterator.batch_size:
        values: [4,]
    model.encode_together:
        values: ["true", ]
    trainer.optimizer.lr:
        values: [0.001,]
    model.dropout:
        values: [0.5]


3. Run training

``wandb sweep wandb_label_predictor1.yaml``

(returns %sweepname1)

``wandb sweep wandb_label_predictor2.yaml``

(returns %sweepname2)

``wandb agent --count 1 %sweepname1 && wandb agent --count 1 %sweepname2``

Move the best model in label_predictor_bimpm

In [ ]:
! ls -laht models/wandb

In [ ]:
! cp -r models/wandb/run-20200709_212429-jyzsnacj/training_dumps models/label_predictor_bimpm/usual-sweep-1

**Or** load from wandb by %sweepname

In [ ]:
import wandb
api = wandb.Api()
run = api.run("tchewik/tmp/7hum4oom")
for file in run.files():
    file.download(replace=True)

In [ ]:
! cp -r training_dumps models/label_predictor_bimpm/light-sweep-1

And run evaluation from shell

``sh eval_label_predictor.sh {elmo|elmo_fasttext} light-sweep-2``

### 4. Evaluate classifier

In [ ]:
def load_predictions(path):
    result = []
    
    with open(path, 'r') as file:
        for line in file.readlines():
            result.append(json.loads(line)["label"])
            
    print('length of result:', len(result))
    return result

In [ ]:
RESULT_DIR = 'light-sweep-1'  # ELMo
RESULT_DIR = 'usual-sweep-1'  # ELMo + fastText

On dev set

In [ ]:
import pandas as pd
import json

true = pd.read_csv(DEV_FILE_PATH, sep='\t', header=None)[0].values.tolist()
pred = load_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_dev.json')

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(true[:len(pred)], pred, digits=4))

In [ ]:
test_metrics = classification_report(true[:len(pred)], pred, digits=4, output_dict=True)
test_f1 = np.array(
    [test_metrics[label].get('f1-score') for label in test_metrics if type(test_metrics[label]) == dict]) * 100

test_f1

In [ ]:
len(true)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(true[:len(pred)], pred, average='macro')*100))
print('pr: %.2f'%(precision_score(true[:len(pred)], pred, average='macro')*100))
print('re: %.2f'%(recall_score(true[:len(pred)], pred, average='macro')*100))

In [ ]:
from utils.plot_confusion_matrix import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

labels = list(set(true))
labels.sort()
plot_confusion_matrix(confusion_matrix(true[:len(pred)], pred, labels), target_names=labels, normalize=True)

In [ ]:
top_classes = [
    'attribution_NS',
    'attribution_SN',
    'purpose_NS',
    'purpose_SN',
    'condition_SN',
    'contrast_NN',
    'condition_NS',
    'joint_NN',
    'concession_NS',
    'same-unit_NN',
    'elaboration_NS',
    'cause-effect_NS',
]

class_mapper = {weird_class: 'other' + weird_class[-3:] for weird_class in labels if not weird_class in top_classes}

In [ ]:
import numpy as np

true = [class_mapper.get(value) if class_mapper.get(value) else value for value in true]
pred = [class_mapper.get(value) if class_mapper.get(value) else value for value in pred]

pred_mapper = {
    'other_NN': 'joint_NN',
    'other_NS': 'joint_NN',
    'other_SN': 'joint_NN'
}
pred = [pred_mapper.get(value) if pred_mapper.get(value) else value for value in pred]

#pred = [value if not 'other' in value else true[i] for i, value in enumerate(pred)]

_to_stay = (np.array(true) != 'other_NN') & (np.array(true) != 'other_SN') & (np.array(true) != 'other_NS')

_true = np.array(true)[_to_stay]
_pred = np.array(pred)[_to_stay[:len(pred)]]
labels = list(set(_true))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(true[:len(pred)], pred, average='macro')*100))
print('pr: %.2f'%(precision_score(true[:len(pred)], pred, average='macro')*100))
print('re: %.2f'%(recall_score(true[:len(pred)], pred, average='macro')*100))

In [ ]:
labels.sort()

In [ ]:
plot_confusion_matrix(confusion_matrix(_true[:len(_pred)], _pred), target_names=labels, normalize=True)

In [ ]:
import numpy as np

for rel in np.unique(_true):
    print(rel)

On train set (optional)

In [ ]:
import pandas as pd
import json

true = pd.read_csv('models/label_predictor_bimpm/nlabel_cf_train.tsv', sep='\t', header=None)[0].values.tolist()
pred = load_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_train.json')

print(classification_report(true[:len(pred)], pred, digits=4))

In [ ]:
file = 'models/label_predictor_lstm/nlabel_cf_train.tsv'
true_train = pd.read_csv(file, sep='\t', header=None)
true_train['predicted_relation'] = pred

print(true_train[true_train.relation != true_train.predicted_relation].shape)

true_train[true_train.relation != true_train.predicted_relation].to_csv('mispredicted_relations.csv', sep='\t')

On test set

In [ ]:
import pandas as pd
import json

true = pd.read_csv(TEST_FILE_PATH, sep='\t', header=None)[0].values.tolist()
pred = load_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_test.json')

print(classification_report(true[:len(pred)], pred, digits=4))

In [ ]:
test_metrics = classification_report(true[:len(pred)], pred, digits=4, output_dict=True)
test_f1 = np.array(
    [test_metrics[label].get('f1-score') for label in test_metrics if type(test_metrics[label]) == dict]) * 100

test_f1

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(true[:len(pred)], pred, average='macro')*100))
print('pr: %.2f'%(precision_score(true[:len(pred)], pred, average='macro')*100))
print('re: %.2f'%(recall_score(true[:len(pred)], pred, average='macro')*100))

In [ ]:
len(true)

In [ ]:
true = [class_mapper.get(value) if class_mapper.get(value) else value for value in true]
pred = [class_mapper.get(value) if class_mapper.get(value) else value for value in pred]
pred = [pred_mapper.get(value) if pred_mapper.get(value) else value for value in pred]

_to_stay = (np.array(true) != 'other_NN') & (np.array(true) != 'other_SN') & (np.array(true) != 'other_NS')

_true = np.array(true)[_to_stay]
_pred = np.array(pred)[_to_stay]

In [ ]:
print(classification_report(_true[:len(_pred)], _pred, digits=4))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(_true[:len(_pred)], _pred, average='macro')*100))
print('pr: %.2f'%(precision_score(_true[:len(_pred)], _pred, average='macro')*100))
print('re: %.2f'%(recall_score(_true[:len(_pred)], _pred, average='macro')*100))

### Ensemble: (Logreg+Catboost) + BiMPM

In [ ]:
model_vocab = [
    "joint_NN",
    "elaboration_NS",
    "contrast_NN",
    "interpretation-evaluation_NS",
    "cause-effect_SN",
    "preparation_SN",
    "attribution_SN",
    "sequence_NN",
    "cause-effect_NS",
    "same-unit_NN",
    "condition_SN",
    "purpose_NS",
    "condition_NS",
    "attribution_NS",
    "background_NS",
    "comparison_NN",
    "evidence_NS",
    "solutionhood_SN",
    "background_SN",
    "concession_NS",
    "purpose_SN",
    "restatement_NN",
    "interpretation-evaluation_SN"
]

catboost_vocab = [
   'attribution_NS', 'attribution_SN', 'background_NS',
   'background_SN', 'cause-effect_NS', 'cause-effect_SN',
   'comparison_NN', 'concession_NS', 'condition_NS', 'condition_SN',
   'contrast_NN', 'elaboration_NS', 'evidence_NS',
   'interpretation-evaluation_NS', 'interpretation-evaluation_SN',
   'joint_NN', 'preparation_SN', 'purpose_NS', 'purpose_SN',
   'restatement_NN', 'same-unit_NN', 'sequence_NN', 'solutionhood_SN']

def load_neural_predictions(path):
    result = []
    
    with open(path, 'r') as file:
        for line in file.readlines():
            probs = json.loads(line)['probs']
            probs = {model_vocab[i]: probs[i] for i in range(len(model_vocab))}
            result.append(probs)
            
    return result

def load_scikit_predictions(model, X):
    result = []
    predictions = model.predict_proba(X)
    
    for prediction in predictions:
        probs = {catboost_vocab[j]: prediction[j] for j in range(len(catboost_vocab))}
        result.append(probs)
    
    return result

def vote_predictions(pred1, pred2, soft=True):
    assert len(pred1) == len(pred2)
    result = []
    
    for i in range(len(pred1)):
        sample_result = {}
        for key in pred1[i].keys():
            if soft:
                sample_result[key] = pred1[i][key] + pred2[i][key]
            else:
                sample_result[key] = max(pred1[i][key], pred2[i][key])
        
        result.append(sample_result)
    
    return result

def probs_to_classes(pred):
    result = []
    
    for sample in pred:
        best_class = ''
        best_prob = 0.
        for key in sample.keys():
            if sample[key] > best_prob:
                best_prob = sample[key]
                best_class = key
        
        result.append(best_class)
    
    return result

In [ ]:
import pickle

fs_catboost_plus_logreg = pickle.load(open('models/relation_predictor_baseline/model.pkl', 'rb'))
lab_encoder = pickle.load(open('models/relation_predictor_baseline/label_encoder.pkl', 'rb'))
scaler = pickle.load(open('models/relation_predictor_baseline/scaler.pkl', 'rb'))
drop_columns = pickle.load(open('models/relation_predictor_baseline/drop_columns.pkl', 'rb'))

On dev set

In [ ]:
from sklearn import metrics


TARGET = 'relation'

y_dev, X_dev = dev_samples[TARGET].to_frame(), dev_samples.drop(TARGET, axis=1).drop(
   columns=drop_columns + ['category_id'])

X_scaled_np = scaler.transform(X_dev)
X_dev = pd.DataFrame(X_scaled_np, index=X_dev.index)

catboost_predictions = load_scikit_predictions(fs_catboost_plus_logreg, X_dev)
neural_predictions = load_neural_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_dev.json')

tmp = vote_predictions(neural_predictions, catboost_predictions)
ensemble_pred = probs_to_classes(tmp)

print('weighted f1: ', metrics.f1_score(y_dev.values, ensemble_pred, average='weighted'))
print('macro f1: ', metrics.f1_score(y_dev.values, ensemble_pred, average='macro'))
print('accuracy: ', metrics.accuracy_score(y_dev.values, ensemble_pred))
print()
print(metrics.classification_report(y_dev, ensemble_pred, digits=4))

On test set

In [ ]:
TARGET = 'relation'

y_test, X_test = test_samples[TARGET].to_frame(), test_samples.drop(TARGET, axis=1).drop(
    columns=drop_columns + ['category_id'])

X_scaled_np = scaler.transform(X_test)
X_test = pd.DataFrame(X_scaled_np, index=X_test.index)

catboost_predictions = load_scikit_predictions(fs_catboost_plus_logreg, X_test)
neural_predictions = load_neural_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_test.json')

tmp = vote_predictions(neural_predictions, catboost_predictions, soft=True)

ensemble_pred = probs_to_classes(tmp)

print('weighted f1: ', metrics.f1_score(y_test.values, ensemble_pred, average='weighted'))
print('macro f1: ', metrics.f1_score(y_test.values, ensemble_pred, average='macro'))
print('accuracy: ', metrics.accuracy_score(y_test.values, ensemble_pred))
print()
print(metrics.classification_report(y_test, ensemble_pred, digits=4))

In [ ]:
test_metrics = classification_report(y_test, ensemble_pred, digits=4, output_dict=True)
test_f1 = np.array(
    [test_metrics[label].get('f1-score') for label in test_metrics if type(test_metrics[label]) == dict]) * 100

test_f1